# Ridge and Lasso Regression - Lab

## Introduction

In this lab, you'll practice your knowledge on Ridge and Lasso regression!

## Objectives

You will be able to:

- Use Lasso and ridge regression in Python
- Compare Lasso and Ridge with standard regression

## Housing Prices Data

Let's look at yet another house pricing data set.

In [46]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Housing_Prices/train.csv')

Look at df.info

In [47]:
# Your code here
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

We'll make a first selection of the data by removing some of the data with `dtype = object`, this way our first model only contains **continuous features**

Make sure to remove the SalesPrice column from the predictors (which you store in `X`), then replace missing inputs by the median per feature.

Store the target in `y`.

In [48]:
# Load necessary packages


# remove "object"-type features and SalesPrice from `X`
X = df.drop('SalePrice',axis=1).select_dtypes(exclude = 'object')
# Impute null values
X.fillna(value = X.median(), inplace = True)
# Create y
y = df.SalePrice

Look at the information of `X` again

In [49]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 37 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtFinSF2       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
LowQualFinSF     1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
BsmtHalfBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
F

## Let's use this data to perform a first naive linear regression model

Compute the R squared and the MSE for both train and test set.

In [65]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import *

def scoring_params(X_train, y_train, X_test, y_test, kind = 'Linear', alpha = 1):
    fit_method = {'linear':LinearRegression(), 'lasso':Lasso(alpha = alpha),'ridge':Ridge(alpha = alpha)}
    
    reg = fit_method[kind.lower()]
    
    reg.fit(X_train, y_train)
    y_hat= reg.predict(X_train)
    y_hat_test = reg.predict(X_test)
    r2 = reg.score(X_train,y_train)
    r2_test = reg.score(X_test, y_test)
    mse = mean_squared_error(y_train, y_hat)
    mse_test = mean_squared_error(y_test, y_hat_test)

    return (r2, r2_test),(mse,mse_test),reg

In [51]:
# Split in train and test
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.25)
# Fit the model and print R2 and MSE for train and test
scoring_params(X_train,y_train,X_test,y_test)

((0.8482181737833239, 0.6069542963010385),
 (980929137.2187454, 2293893752.287989))

## Normalize your data

We haven't normalized our data, let's create a new model that uses `preprocessing.scale` to scale our predictors!

In [54]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
# Scale the data and perform train test split
X_train_scaled = scaler.fit_transform(X_train)
X_train_scaled = pd.DataFrame(data = X_train_scaled, columns = X_train.columns)
X_test_scaled = scaler.fit_transform(X_test)
X_test_scaled = pd.DataFrame(data = X_test_scaled, columns = X_train.columns)
X_train_scaled.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,...,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000,1095.000000
mean,0.500853,0.217862,0.300207,0.043752,0.567529,0.568950,0.721441,0.582877,0.063937,0.195117,...,0.337651,0.112397,0.085694,0.055599,0.006882,0.030320,0.003757,0.004619,0.491407,0.454795
std,0.285700,0.248077,0.124086,0.052119,0.156173,0.137245,0.219345,0.343174,0.114745,0.194177,...,0.147762,0.148071,0.122729,0.154272,0.060158,0.117029,0.055951,0.038885,0.247336,0.333788
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.257721,0.000000,0.242236,0.028979,0.444444,0.500000,0.594203,0.283333,0.000000,0.000000,...,0.241727,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.363636,0.250000
50%,0.498284,0.176471,0.298137,0.037776,0.555556,0.500000,0.746377,0.733333,0.000000,0.168584,...,0.343885,0.000000,0.043876,0.000000,0.000000,0.000000,0.000000,0.000000,0.454545,0.500000
75%,0.745710,0.294118,0.360248,0.047996,0.666667,0.625000,0.931159,0.900000,0.102188,0.315265,...,0.414388,0.196033,0.127971,0.000000,0.000000,0.000000,0.000000,0.000000,0.636364,0.750000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Perform the same linear regression on this data and print out R-squared and MSE.

In [55]:
# Your code here
scoring_params(X_train_scaled,y_train,X_test_scaled,y_test)

((0.8482181737833243, 0.4636998255788679),
 (980929137.2187433, 3129955646.068659))

## Include dummy variables

We haven't included dummy variables so far: let's use our "object" variables again and create dummies

In [56]:
# Create X_cat which contains only the categorical variables
X_cat = df.select_dtypes(include = 'object')

In [57]:
# Make dummies
X_cat = pd.get_dummies(X_cat)
X_cat.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0


Merge `x_cat` together with our scaled `X` so you have one big predictor dataframe.

In [58]:
# Your code here
X_train_scaled_merged = X_train_scaled.join(X_cat)
X_test_scaled_merged = X_test_scaled.join(X_cat)

Perform the same linear regression on this data and print out R-squared and MSE.

In [59]:
# Your code here
scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged,y_test)

((0.8776967230099345, -2.338893903538857e+19),
 (790416421.8292238, 1.365025507746423e+29))

Notice the severe overfitting above; our training R squared is quite high, but the testing R squared is negative! Our predictions are far far off. Similarly, the scale of the Testing MSE is orders of magnitude higher then that of the training.

## Perform Ridge and Lasso regression

Use all the data (normalized features and dummy categorical variables) and perform Lasso and Ridge regression for both! Each time, look at R-squared and MSE.

## Lasso

With default parameter (alpha = 1)

In [66]:
# Your code here
scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged
               ,y_test, kind = 'lasso', alpha = 1)

((0.8776682184458909, 0.4055238203781971),
 (790600639.9145576, 3469482509.2476616),
 Lasso(alpha=1, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False))

With a higher regularization parameter (alpha = 10)

In [67]:
# Your code here
scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged
               ,y_test, kind = 'lasso', alpha = 10)

((0.8763912457416021, 0.4002107398875414),
 (798853404.8488903, 3500490715.2354646),
 Lasso(alpha=10, copy_X=True, fit_intercept=True, max_iter=1000,
    normalize=False, positive=False, precompute=False, random_state=None,
    selection='cyclic', tol=0.0001, warm_start=False))

## Ridge

With default parameter (alpha = 1)

In [68]:
# Your code here
scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged
               ,y_test, kind = 'ridge', alpha = 1)

((0.8749671370228436, 0.43315205763060616),
 (808057074.1656846, 3308238561.7280827),
 Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001))

With default parameter (alpha = 10)

In [69]:
# Your code here
scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged
               ,y_test, kind = 'ridge', alpha = 10)

((0.8513437746433108, 0.507880465055305),
 (960729136.7885209, 2872108551.1549616),
 Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None,
    normalize=False, random_state=None, solver='auto', tol=0.001))

## Look at the metrics, what are your main conclusions?

Ridge performed the best with the test set at an alpha level of 10 for the data with every predictor. The data with fewer predictors performed better, actually, than everything else.

## Compare number of parameter estimates that are (very close to) 0 for Ridge and Lasso

In [78]:
# number of Ridge params almost zero
_,_,ridge_reg = scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged
               ,y_test, kind = 'lasso', alpha = 10)
len(ridge_reg.coef_>0.05)

289

In [77]:
# number of Lasso params almost zero
_,_,lasso_reg = scoring_params(X_train_scaled_merged,y_train,X_test_scaled_merged
               ,y_test, kind = 'lasso', alpha = 10)
sum(lasso_reg.coef_>0.05)

116

Compare with the total length of the parameter space and draw conclusions!

In [79]:
# your code here
len(X_test_scaled_merged.columns)

289

## Summary

Great! You now know how to perform Lasso and Ridge regression.